# import notebook

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import io,os,sys,types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
sys.meta_path.append(NotebookFinder())


In [2]:
#import chr_word_to_vec
import crf_gru_model

importing Jupyter notebook from crf_gru_model.ipynb


In [3]:
import numpy as np
import re
import json
import torch.utils.data
from torch.utils.data import DataLoader
import io,os
from torch import nn
from gensim.models import word2vec
from torch.nn.utils.rnn import pad_sequence
from torch import optim
from torch.autograd import Variable
from torchtext import data

In [4]:
def cal_dialogue(data):
    '''
    used in lamda function
    
    return longest sentence len per dialogue
    '''
    i = 0
    maxleng = 0
    while(i < len(data.Text)): # len(data.Text) = dialogue length
        if len(data.Text[i]) > maxleng:
            maxleng = len(data.Text[i])
        i = i + 1
    return maxleng

In [5]:
my_fields={'dial': ('Text', data.Field(sequential=True)),
        'emo': ('labels_1', data.Field(sequential=False)),
        'act': ('labels_2', data.Field(sequential=False))}

BATCH_SIZE = 100
device = torch.device("cuda")

tag_to_ix = {'start_tag':0,'stop_tag':29,'pad_tag':30}

working_path = '/home/jongsu/jupyter/pytorch_dialogue_ie/'
WV_PATH = '/home/jongsu/jupyter/pytorch_dialogue_ie/parameter/dialogue_wv'

train = crf_gru_model.my_TabularDataset.splits(path = working_path, train = 'data_jsonfile/full_data.json',
                          fields=my_fields) 
train = sorted(train, key = lambda  x: cal_dialogue(x))
train = train[:-1118] #exclude dialogue which has extremely long sentence (0~11117 => 0~9999)
train = sorted(train, key = lambda  x: -len(x.Text)) #reordering training dataset with number of sentences
# low index has much sentence because afterwards we use torch pad_sequence
dataseq = torch.arange(end = len(train),dtype=torch.int)

In [6]:
sent_net = crf_gru_model.makesent_gru(100, True).cuda()
#we need convert batch*dialogue -> batch*(dialogue_length*sent_vec(float))

my_grucrf_model = crf_gru_model.BiGru_CRF(tag_to_ix,BATCH_SIZE).cuda()
#this convert batch*(dialogue_length*sent_vec(float)) -> batch*(dialogue_tag,score)

In [7]:
def load_models(path1,path2):
    sent_to_vextor_bigru_net.load_state_dict(torch.load(path1))
    my_grucrf_model.load_state_dict(torch.load(path2))
    return

load_models(working_path + '/parameter/shared.pth',working_path+'/parameter/crf_gru.pth')

In [8]:
errary = []
errary = crf_gru_model.train_func(
    train_data = train, 
    shared_model = sent_net,
    comp_model = my_grucrf_model,
    dataseq = dataseq, 
    filtering_value = 3, 
    iter_num = 1, 
    batch_size = 100,
    learning_rate = 0.00003)



batchnum =  100
###############################################errcount 93
1
tag =  tensor([  0,   2,   1,  26,   2,   1,   3,  20,   1,   1,   2,   1,
          1,   1,   1,   2,   1,  18,   1,   3,   4,  18,   1,   2,
          3,   3,   3,   8,   1,  29,  30,  30,  30,  30], device='cuda:0')
expect =  [2, 3, 3, 2, 1, 1, 4, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 3, 3, 2, 1, 3, 3, 1, 3, 1, 1, 29]
accuracy =  0.7678571428571428
tensor([ 103.0568,   39.6829,   60.6473,   27.1775,   62.3598,  161.5406,
         118.9767,  116.2921,   69.0526,  171.6000,  154.5450,   55.0343,
          62.7466,   71.2227,   49.0184,   13.9861,  458.5493,   22.4053,
          12.3049,   48.1712,   40.0836,  129.1522,  154.7709,   42.2505,
          90.3113,   29.4835,  142.0879,  192.6058,   81.3536,  126.5451,
         149.6495,   91.7379,   27.5388,   39.9140,   67.0382,   78.7792,
          42.4706,   29.2948,   40.4326,   29.8271,   19.9142,    7.6335,
          72.8914,  124.4497,   49.0323,   76.7361,   52

KeyboardInterrupt: 

In [9]:
'''
retraining with earray
dataseq = torch.tensor(errary, dtype=torch.int)
newary = train_func(train, sent_to_vextor_bigru_net, my_grucrf_model, dataseq, 20, 1)
'''

'\nretraining with earray\ndataseq = torch.tensor(errary, dtype=torch.int)\nnewary = train_func(train, sent_to_vextor_bigru_net, my_grucrf_model, dataseq, 20, 1)\n'